<div>
<img src='../img/WSP_red.png' style='height: 95px; float: left' alt='WSP Logo'/>
<img src='../img/austroads.png' style='height: 115px; float: right' alt='Client Logo'/>
</div>
<center><h2>AAM6201 Development of Machine-Learning Decision-Support tools for Pavement Asset Management<br>Case Study 2: Funding Allocation
</h2></center>


In [ ]:
import time
import datetime
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import differential_evolution

from data import DATA_DIR
from pathlib import Path
from src.config import CONFIG
from src.util import load_data, get_objective_value, var_dict_to_series
from src.inner_strategy import early_stop
from src.forms.ip_form import make_problem_penalties

Plot util

In [ ]:
def plot_output_plane(
    split_x: np.ndarray, 
    split_y: np.ndarray, 
    los_z: np.ndarray,
    sequence_color: np.ndarray,
    x_label: str='X-Axis',
    y_label: str='Y-Axis',
    z_label: str='Z-Axis',
    color_label: str='Sequence',
    **kwargs
):
    """
    Plot 3d plot given values of budget splits for 2 criteria on the x and y axis, 
    as well as their corresponding level of service on z axis
    """    
    assert split_x.ndim == 1 and split_y.ndim == 1 and los_z.ndim == 1 and sequence_color.ndim == 1
    assert len(split_y) == len(split_x) and len(split_y) == len(los_z) == len(sequence_color)

    fig = go.Figure(data=[go.Scatter3d(
        x=split_x,
        y=split_y,
        z=los_z,
        mode='markers',
        marker=dict(
            color=sequence_color,
            colorbar=dict(
                title=dict(
                    text=color_label
                )
            ),
        ),
        **kwargs
    )])

    fig.update_layout(
        autosize=False,
        width=1000,
        height=800,
        title='Level of Service against Splits',
        scene=dict(
            yaxis_title=y_label,
            xaxis_title=x_label,
            zaxis_title=z_label,
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )
    return fig

# Outer optimisation for optimal penalties

In [ ]:
def outer_objective(x, *args):
    """Function to evaluate the inputs x given by the optimiser. A list is passed in to capture the history of changes"""
    df_pickle = args[0]
    seq = args[1]
    df = pickle.loads(df_pickle)
    pen_dict = {CONFIG['metro_penalty_col']: x[0], CONFIG['freight_penalty_col']: x[1]}
    (status, _, var_dict, objective), verifications = early_stop(df, make_problem_penalties, patience=3, penalties=pen_dict, budget=float(CONFIG['budget']))
    obj_dict = get_objective_value(df, var_dict_to_series(var_dict))
    seq.append({**obj_dict, **pen_dict})
    return -obj_dict["Absolute_LoS"]

In [ ]:
df = load_data()
df_pickle = pickle.dumps(df)
bounds = [(-100, 100)] * 2
sequence = []
start = time.time()
opt_res = differential_evolution(outer_objective, bounds=bounds, args=(df_pickle, sequence), seed=1, disp=True)

print("Finished in {}".format(datetime.timedelta(seconds=time.time()-start)))

## Process result

In [ ]:
# run the optimisation problem with found optimal penalties
df = pickle.loads(df_pickle)
pen_dict = {CONFIG['metro_penalty_col']: opt_res.x[0], CONFIG['freight_penalty_col']: opt_res.x[1]}
(status, _, var_dict, objective), verifications = early_stop(df, make_problem_penalties, patience=3, penalties=pen_dict, budget=float(CONFIG['budget']))
obj_dict = get_objective_value(df, var_dict_to_series(var_dict))

In [ ]:
seq_df = pd.DataFrame.from_records(sequence) # sequence stores output of the inner 
x = seq_df["metro_perc"]
y = seq_df["freight_perc"]
z = seq_df["Absolute_LoS"]
color = seq_df.index

# Absolute LoS by percentage
fig = plot_output_plane(x, y, z, color, x_label='Metro Percentage', y_label='Freight Percentage', z_label='Level of Service', color_label='Iteration')
fig.show()

In [ ]:
x = seq_df["Metro"]
y = seq_df["Freight"]
z = seq_df["Absolute_LoS"]
color = seq_df.index

# Absolute LoS by penalties
fig = plot_output_plane(x, y, z, color, x_label='Metro Penalty', y_label='Freight Penalty', z_label='Level of Service', color_label='Iteration')
fig.show()

In [ ]:
# plot -LoS  as function of time (iteration)
(-seq_df["Absolute_LoS"]).plot(figsize=(12, 12), xlabel="Iteration", ylabel="Negative Absolute LoS", title="Differential Evolution Convergence")
plt.savefig(Path('.').resolve().parent / "reports" / "figures" / "de_convergence.png")